In [2]:
from nltk.tag.stanford import POSTagger
import string
english_postagger = POSTagger('./stanford-postagger-2015-04-20/models/english-bidirectional-distsim.tagger', './stanford-postagger-2015-04-20/stanford-postagger.jar')

In [10]:
f = open('./text', 'r')
f2 = open('./result', 'r')
name = []

for line in f2:
    name.append(line.split('      ')[0])
    
f2.close()

keyword = ['brother']

keyphrase = []  #without punctuations
original = []    # with punctuations

for line in f:
    if keyword[0] in line:
        original.append(line.replace(' ',''))
        temp = ""
        for l in line:
            if l not in string.punctuation:
                temp = temp.join(l)
            else:
                temp = temp.join(' ')
        keyphrase.append(temp)

order = 0
for c in keyphrase:
    n = 0
    position = 0
    tokens = english_postagger.tag(c.split())
    #print tokens
    for element in tokens[0]:
        if keyword[0] in element:
            condition = ''         # ab for both behind and forword, abc without puctuations
            position = n
            #print position
            for i in range(position-1,-1,-1):
                if 'NN' in tokens[0][i] or 'JJ' in tokens[0][i]:
                    pass
                elif 'PRP$' in tokens[0][i]:
                    condition = condition + 'a'
                    prp = tokens[0][i][0]
                    start = i
                    break
                else:
                    break
            relation = keyword[0]
            if position+1 != len(tokens[0]):
                for i in range(position+1,len(tokens[0])):
                    if 'NNP' not in tokens[0][i]:
                        if position+1 != i:
                            condition = condition + 'b'
                        for j in range(position+1,i):
                            name = tokens[0][j][0]
                        end = i
                        break
            else:
                end = position

        n = n + 1
        

    temp = ''
    print tokens
    for i in range(start,end):
        print start
        print end
        print i
        temp = temp + tokens[0][i][0]
    if temp in original[order]:
        condition = condition + 'c'
    print condition
    if condition == 'abc':
        print prp + '   ' + relation + '   ' + name + '\n'
    else:
        print 'result doesn\'t match\n'
        
    order = order + 1
        
    
f.close()



[[]]
13
16
13


IndexError: list index out of range